In [1]:
import re
import numpy as np

### Stworzenie słownika z bazy sjp

In [2]:
with open ("files/polish.stopwords.txt", "r", encoding="utf8") as file:
    lista = file.read()
lista_sp = []
for el in lista.split():
    if len(el) >=3:
        lista_sp.append(el)

In [3]:
D = {}
cores = []
max_length = 10
min_length = 4
with open("files/odm.txt", "r", encoding="utf8") as f:
    while words_list := f.readline():
        words = words_list.replace(", ", ",").replace("\n", "")
        found = False
        for sign in ".:- !?":
            if sign in words:
                found = True
        words_list = words.split(",")
        core = words_list[0].lower()
        if found or len(core) > max_length or len(core) < min_length or core in lista_sp:
            continue 
        cores.append(core)
        for word in words_list:
            word = word.lower()
            if word not in D:
                D[word] = core         
cores = list(set(cores))

In [4]:
len(cores)

138163

In [6]:
# lista_sp

### Funkcja do lematyzacji
(kto by się spodziewał)

In [16]:
def clean(tekst):
    res = ""
    tekst = tekst.replace('\n', ' ')
    tekst = tekst.lower()
    for el in tekst:
        if el not in [".",",","?","!",";",":","%", "&", ")",">","]","$","@","(", "[", "<", "0","1","2","3","4","5","6","7","8","9"]:
            res += el 
    return res

def lemmatisation(text, D, stopwords):
#     text = re.sub("[,.:]", "", text)
#     text = text.replace("\n", " ")

    text = clean(text)
#     print(text)
#     return " ".join([D[word] if word in D else word for word in text.split(" ") ])
    return " ".join([D[word] for word in text.split(" ") if word in D and word not in stopwords])    

In [17]:
text = """Po pierwsze przekazać swój protest w związku z naruszeniem polskiej przestrzeni powietrznej oraz dodatkowo w związku z zachowaniem ambasadora drogą pisemną, czyli za pomocą noty dyplomatycznej, którą możemy przesłać zarówno Ambasadzie Rosji w Warszawie, jak i dla zwiększenia efektu poprzez naszą ambasadę rosyjskiemu MSZ.
Po drugie można wydać dyspozycję ambasadorowi RP w Moskwie, by ten równie ostentacyjnie, co jego rosyjski odpowiednik w Warszawie, zignorował jedno z najbliższych zaproszeń do rosyjskiego MSZ, a następnie udzielił wywiadu wybranemu polskiemu medium, w którym wyjaśni, że nie odpowiadała mu forma zaproszenia.
Trzecia opcja, na którą wydawał się niestety wskazywać rzecznik MSZ Paweł Wroński, to wydalenie ambasadora Rosji. Problem polega na tym, że to, na co wskazywał rzecznik MSZ, opłaca się Rosji, ale nie Polsce. Jeśli bowiem Polska wydali ambasadora Rosji, to Rosjanie w ramach wzajemności, czyli zasady dyplomacji, którą bezwzględnie stosują, wydalą z Rosji ambasadora RP w Moskwie Krzysztofa Krajewskiego. Byłoby to głęboko niezgodne z żywotnymi interesami narodowymi RP.
Utrzymywanie kanału komunikacji z Moskwą na poziomie ambasadora jest w sytuacji toczącej się za naszą granicą wojny bardziej, a nie mniej istotne, niż w czasach pokoju.
Co więcej, obecność ambasadora pozwala wnikliwej monitorować to, co dzieje się pomiędzy np. Waszyngtonem a Moskwą. Mówiąc wprost, jeśli założyć, że Amerykanie nie zawsze lub nie od razu nam wszystko mówią, to warto mieć nasłuch nie tylko w Departamencie Stanu USA czy też w Białym Domu, ale również Ambasadzie USA w Moskwie.
W dodatku obecny ambasador w Moskwie należy do wąskiego grona ambasadorów państw NATO w Moskwie, których głos dociera nie tylko do ich stolic, ale również do Kwatery Głównej paktu."""
new_text = lemmatisation(text, D, lista_sp)
new_text

'pierwsza przekazać protest związek naruszyć polski przestrzeń powietrzny dodatkowy związek zachować ambasador droga pisemny pomoc nota przesłać zarówno ambasada rosja warszawa zwiększyć efekt poprzez ambasada rosyjski druga wydać dyspozycja ambasador moskwa równia rosyjski warszawa zignorować najbliższy zaprosić rosyjski następny udzielić wywiad wybrać polski medium wyjaśnić odpowiadać forma zaprosić trzeci opcja wydawać niestety wskazywać rzecznik paweł wroński wydalić ambasador rosja problem polegać wskazywać rzecznik opłacać rosja polska polska wydać ambasador rosja rosjanin rama wzajemność zasada dyplomacja stosować wydalić rosja ambasador moskwa krzysztof krajewski głęboki zgodny żywotny interes narodowy utrzymywać kanał moskwa poziom ambasador sytuacja toczyć granica wojna mniej istotny czas pokój obecność ambasador pozwalać wnikliwy dziać pomiędzy waszyngton moskwa mówić wprost założyć amerykan mówić warta miecia nasłuch stan biali ambasada moskwa dodatek obecny ambasador moskw

### Zliczanie powtórzeń

In [10]:
occurences = {}
for word in new_text.split(" "):
    if word not in occurences:
        occurences[word] = 1
    else:
        occurences[word] += 1

In [11]:
# {word:occurences[word] for word in occurences.keys() if occurences[word] > 1}
sorted(((occurence, word) for word, occurence in occurences.items()), reverse=True)[:4]

[(9, 'ambasador'), (7, 'moskwa'), (5, 'rosja'), (3, 'rosyjski')]

#### todo
- pod prezentację stworzyć histogram
- wyrzucić spójniki
- rozdzielic na slownik z rzeczownikami i z czasownikami


In [90]:
D["egzekucja"]

'egzekucja'

Dzisiaj przeprowadzono egzekucję na krześle elektrycznym

### Bag-of-words

In [12]:
len(cores)

138163

In [13]:
def vectorize(sentence_L, all_words):
    vector = np.zeros(len(all_words))
    for word in sentence_L:
        if word in all_words:
            idx = all_words.index(word)
            vector[idx] += 1
    return vector

In [14]:
text_L = new_text.split(" ")

In [15]:
X = vectorize(text_L, cores)

In [110]:
X

array([0., 0., 0., ..., 0., 0., 0.])

###### Zdanie
Marcinowi i o rok młodszemu Krzyśkowi ciężko bronić się podczas procesu. Konsekwentnie mówią, że z zabójstwem braci Nefel nie mają nic wspólnego. Nie było ich na miejscu zbrodni. Gdy doszło do morderstwa, byli ze swoimi rodzinami. Ich zeznania są spójne. Nie zmieniają się.

Kontekst `Zbrodnia`

In [22]:
with open('files/zdania.txt', 'r', encoding='utf8') as f:
    sentences = []
    sentence = ""
    count_breaks = 0
    while line := f.readline():
        if line == '\n':
            count_breaks += 1
        else:
            sentence += line
        if count_breaks == 2:
            sentences.append(sentence)
            sentence = ""
            count_breaks = 0
        

In [23]:
len(sentences)

10

In [25]:
lemma = lemmatisation(sentences[0], D, lista_sp)
print(lemma)
vec = vectorize(lemma, cores)
vec

ochronny markiz tarasowy markiz boczny chronić ciekawska spojrzeć sąsiad dzięki zakłócenia śniadać taras ponadto dziecko uchwyt sprawiać markiz zawodny opcja instalacja użyć dołączyć uchwyt podłogowy przykręcić jeść podłoże wsunąć słupek boczny uzyskać stała instalacja należeć przykręcić słupek boczny podłoże wytrzymać odporny rdza słupek markiz wykonać wysoka jakość stabilny metal powłoka proszkowo sprawiać rdzewieć rozkładać


array([0., 0., 0., ..., 0., 0., 0.])